# 2D test mapping from position $x$ to position $y$ and $\delta q$ with NN

In [ ]:
import numpy as np
# %load_ext tensorboard
import time    

import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.contrib.distributions as ds
from IPython.core import display
import data_generation as dg
plt.style.use('ggplot')
%matplotlib inline

# https://github.com/moble/quaternion  install quaternion and numba
import quaternion # [w,x,y,z] order


import mapping_tools as tools

# use conda env 'tf1'

%load_ext autoreload 
%autoreload 2

In [ ]:
sess = tf.InteractiveSession()

## prepare data X and Y

## spherical linear interpolation to generate data for straight lines


In [ ]:
# create grid_lines for visualization
downsampling = 10
nbGrid = 20 * downsampling + 1
def generate_grid(xk, xk2, offset = 1.5):
    grid_lines=[]
    x_grid_ = np.linspace(np.min(xk[:,0]) - offset, np.max(xk[:,0]) + offset, nbGrid)
    y_grid_ = np.linspace(np.min(xk[:,1]) - offset, np.max(xk[:,1]) + offset, nbGrid)
    x_grid, y_grid = np.meshgrid(x_grid_, y_grid_)
    for ii in range(nbGrid):
        for jj in range(nbGrid):               
            x_grid_point = np.array([x_grid[ii,jj],y_grid[ii,jj] ])
            grid_lines.append(x_grid_point)

    grid_lines = np.asarray(grid_lines)
    return grid_lines


In [ ]:
# set sampling points for each object
sp = 100 # 1,10,100 
obj = dg.Shape

# generate test objects
p0 = obj(sample_points=sp, type=0,translation=np.array([0.7,0.9]),scale=0.1)
p1 = obj(sample_points=sp, type=1,translation=np.array([0,1]),scale=0.15)
p2 = obj(sample_points=sp, type=2,translation=np.array([1,0]),scale=0.15)
p3 = obj(sample_points=sp, type=3,translation=np.array([1.2,0.5]),scale=0.15)
p4 = obj(sample_points=sp, type=4,translation=np.array([0,0]),scale=0.5)


x_data = np.concatenate([p0.xy,p1.xy,p2.xy,p3.xy,p4.xy],axis=0)
x_key = [p0.key_points,p1.key_points,p2.key_points,p3.key_points,p4.key_points]
x_data = np.concatenate([x_data, np.repeat([[1.,0,0,0]],5*sp,axis=0)],axis=1)
fig = plt.figure(figsize=(10,10./2))
ax = fig.add_subplot(111)
ax.set_aspect("equal")
ax.scatter(x_data[:,0], x_data[:,1],s=1)
xk = np.concatenate([p0.xy[0,None,:],p1.xy[0,None,:],p2.xy[0,None,:],p3.xy[0,None,:],p4.xy[0,None,:]],axis=0)
ax.set_title('Objects on the local side')
# print xk
fig = plt.figure(figsize=(10,10./2))

ax = fig.add_subplot( 111,sharey=ax)
# sharey=ax
ax.set_aspect("equal")

# set the rotation angle on objects
angle = [0,-30,60,30,30.]
p0 = obj(sample_points=sp, type=0,translation=np.array([0.7,0.7])+np.array([0.8,0.5]),theta=angle[0],scale=0.1)
p1 = obj(sample_points=sp, type=1,translation=np.array([0,1])+np.array([0.4,0.2]),theta=angle[1],scale=0.15)
p2 = obj(sample_points=sp, type=2,translation=np.array([1,0])+np.array([0.7,0.2]),theta=angle[2],scale=0.15)
p3 = obj(sample_points=sp, type=3,translation=np.array([1.5,0.8])+np.array([0.5,0.3]),theta=angle[3],scale=0.15)
p4 = obj(sample_points=sp, type=4,translation=np.array([0,0])+np.array([0,0.2]),theta=angle[4],scale=0.5)
xk2 = np.concatenate([p0.xy[0,None,:],p1.xy[0,None,:],p2.xy[0,None,:],p3.xy[0,None,:],p4.xy[0,None,:]],axis=0)
y_key = [p0.key_points,p1.key_points,p2.key_points,p3.key_points,p4.key_points]

p = [p0.xy,p1.xy,p2.xy,p3.xy,p4.xy]

q_all =np.zeros([5,4])
for i in range(5):
    q = quaternion.from_euler_angles([-angle[i]*np.pi/180.,0,0])
    q_tmp = quaternion.as_float_array(q)
    q_all[i,:] = q_tmp

    p[i] = np.concatenate([p[i],np.repeat(q_tmp.reshape(1,-1), sp,axis=0)],axis=1)
xk2 = np.concatenate([xk2, q_all],axis=1)
y_data = np.concatenate([p[0],p[1],p[2],p[3],p[4]],axis=0)
ax.scatter(y_data[:,0], y_data[:,1],s=1)
ax.set_title('Objects on the remote side')

In [ ]:
delta_quaternion = quaternion.from_float_array(y_data[:,2:])/ quaternion.from_float_array(x_data[:,2:])
delta_q = quaternion.as_float_array(delta_quaternion)
delta_q_tan = tools.log_q(delta_q)


In [ ]:
# to evaluate the jacobian
def batch_jacobians(ys, xs):
    """
    ys : [None, n_y] or [n_y]
    xs : [None, n_x] or [n_x]
    """
    if ys.shape.ndims == 2:
        return tf.transpose(
            tf.stack([tf.gradients(ys[:, i], xs)[0] for i in range(ys.shape[-1].value)]),
            (1, 0, 2))
    elif ys.shape.ndims == 1:
        return tf.stack([tf.gradients(ys[i], xs)[0] for i in range(ys.shape[0].value)])
    else:
        raise NotImplementedError

In [ ]:
def gen_uniform_samples(low, high, size):
    if low.shape[0]==1:
        output = (high - low) * np.random.random_sample((size,)) + low
    elif low.shape[0]==2:
        output_x = (high[0] - low[0]) * np.random.random_sample((size,)) + low[0]
        output_y = (high[1] - low[1]) * np.random.random_sample((size,)) + low[1]
        output = np.transpose(np.vstack([output_x, output_y]))
    elif low.shape[0]==3:
        output_x = (high[0] - low[0]) * np.random.random_sample((size,)) + low[0]
        output_y = (high[1] - low[1]) * np.random.random_sample((size,)) + low[1]
        output_z = (high[2] - low[2]) * np.random.random_sample((size,)) + low[2]
        output = np.transpose(np.vstack([output_x, output_y, output_z]))
    else:
        raise Exception("Invalid dimension: ", low.ndim )
    return output



In [ ]:
def tmp_create_samples(x,num):
    
    # x: [N,7]
    # a_re: [n, 7]
    
    center = np.mean(x[:,:2],axis=0)
    # aa = np.random.multivariate_normal(center, 0.15* np.eye(2), (num, )) # random samples
    offset_2 = 1.5
    aa = gen_uniform_samples(center-offset_2, center+offset_2, num)
    r = 0
    a_re = []
    weight = []
    for i in range(num):
        dis2 = np.sqrt( np.sum((aa[i,:] - center)**2) )
        if dis2 >r:
            a_re.append(aa[i,:] )
            weight.append(dis2-r)
    if len(weight)==0:
        pass
        return a_re, weight/ (sum(weight)/len(weight) )
    else:
        return a_re, weight/ (sum(weight)/len(weight) )

samples, w_samples= tmp_create_samples(x_data[:,:2],200)
s2 = np.asarray(samples)
# # print s2

fig = plt.figure(figsize=(10.,10.))
ax = fig.add_subplot(111)

ax.plot(x_data[:, 0], x_data[:, 1],'k-')
ax.axis('equal')
ax.scatter(s2[:,0], s2[:,1], s = 10)

In [ ]:
def add_layer(inputs, in_size, out_size, name='hidden_layer', scale=1.0,reuse=False, activation_function=None): #activation_function=None
    with tf.variable_scope(name) as scope:
        if reuse:
            scope.reuse_variables()
        w_reg=tf.contrib.layers.l2_regularizer(scale)
        Weights =  tf.get_variable(name='w', initializer=tf.random_normal([in_size,out_size]), regularizer=w_reg)#Weight
        biases = tf.get_variable(name='b', initializer=tf.Variable(tf.zeros([1,out_size])+0.1) )#biases
        # print Weights.name, biases.name
        Wx_plus_b = tf.matmul(inputs, Weights)+biases #inputs*Weight+biases
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        return outputs

In [ ]:
def x2q_NN(x, mid_channel, reuse=False, name ='l', scale= 1., activation=tf.nn.tanh  ):
    x = add_layer(x, 2, mid_channel, reuse=reuse, name= name+'1', scale=scale, activation_function=activation  )
    x = add_layer(x, mid_channel, mid_channel,reuse=reuse, name=name+'2',scale=scale, activation_function=activation  )
    x = add_layer(x, mid_channel, mid_channel,reuse=reuse, name=name+'3', scale=scale, activation_function=activation  )
    # x = add_layer(x, mid_channel, mid_channel,reuse=reuse, name=name+'31',scale=scale,  activation_function=activation  )
    x = add_layer(x, mid_channel, 3, reuse=reuse, name=name+'4',scale=scale, activation_function=activation  )
    return x
# generate weight for cost_pred 
def generate_pred_weight(p_num, lines_num,w1):
    a = np.empty([0])
    for i in range(p_num-1):          
        a1 = np.array([w1])
        a2 = np.ones(lines_num - 2)*1.0
        a3 = np.concatenate([a1,a2,a1])
        a = np.concatenate([a,a3])
#     a4 = a4/np.linalg.norm(a4)
    return a
nbPts = xk.shape[0]
w_test = generate_pred_weight(nbPts, 2, 10.)
print w_test.shape

In [ ]:
# create placeholder to evaluate the NN
x_ph = tf.placeholder(tf.float32, (None, 2), name = 'x_ph')
y_ph = tf.placeholder(tf.float32, (None, 2), name = 'y_ph')
q_tan_ph = tf.placeholder(tf.float32, (None, 3), name = 'q_tan_ph')
q_ph = tools.exp_tf(q_tan_ph) # exp



$\text{log}(\overrightarrow q)= \text{log}(v,\overrightarrow u)=\left\{
\begin{aligned}
&\text{arccos}^{*}(v) \frac{\overrightarrow u}{||\overrightarrow u||},  &||\overrightarrow u ||\ne 0\\
&[0, 0, 0]^T,  & \text{else} \\
\end{aligned}
\right. $

In [ ]:
from inv_net import real_nvp


In [ ]:
activation = tf.nn.tanh 
mid_channel = 24
scale = 0.3


x_jac = tf.placeholder(tf.float32, (None, 2),name='x_jac')
weight_jac = tf.placeholder(tf.float32, (None, ),name='weight_jac')
xk_weight = tf.placeholder(tf.float32, (None, ),name='xk_weight')


q_tan_pred = x2q_NN(x_ph, mid_channel, scale=scale, activation=activation) # hidden layer
q_pred = tools.exp_tf(q_tan_pred)

q_tan_jac = x2q_NN(x_jac, mid_channel, reuse=True, scale=scale, activation=activation) # hidden layer
q_jac = tools.exp_tf(q_tan_jac)

In [ ]:
scale_kernel= .5
mid_channel = 24
y_nn = real_nvp(x_ph, mid_channel, backward=False, reuse=False, name='nvp_0', activation=activation, scale_kernel=scale_kernel) 

# create inverse evaluation
y_in = tf.placeholder(tf.float32, (None, 2),name='y_in')
x_rec = real_nvp(y_in, mid_channel, backward=True, reuse=True, name='nvp_0', activation=activation, scale_kernel=scale_kernel) 


In [ ]:
# placeholder to evaluate the jacobian
y_jac = real_nvp(
    x_jac, mid_channel, backward=False, name='nvp_0' ,
    reuse=True, activation=activation, scale_kernel=scale_kernel) 
jac_xy = batch_jacobians(y_jac, x_jac)



In [ ]:

q_identity = tf.constant([1.0, 0, 0, 0])

tmp_angle = 2* tf.reduce_sum (q_jac * q_identity, axis=1)**2  - 1 
# tmp_angle2 = tf.clip_by_value(tmp_angle,0,1)
# delta_dis = tf.acos(tmp_angle2)
# cost_jac = tf.reduce_sum(delta_dis*weight_jac)
jac_samples_num = 200
cost_jac_xq =  jac_samples_num - tf.reduce_sum(tmp_angle * weight_jac)


In [ ]:

# cost_pred = tf.reduce_sum(orientation_distance(q_pred, q_ph))

cost_pred_xq = x_data.shape[0] - tf.reduce_sum(2 * tf.reduce_sum(q_pred * q_ph, axis=1)**2 - 1)
scale_jac = tf.Variable(1.4)
scale_sig = tf.sigmoid(scale_jac)*2

cost_jac_xy = tf.reduce_sum( tf.reduce_sum((jac_xy -  scale_jac*tf.eye(2)) ** 2,[1,2]) * weight_jac )

cost_pred_xy = tf.reduce_sum( tf.reduce_sum( (y_ph - y_nn) ** 2,axis=1)* xk_weight )

In [ ]:
lmbda_jac = tf.placeholder(tf.float32, (4, ),name='lmbda_jac')   # [cost weight, smooth weight 0.95]

condition = tf.placeholder(tf.int32, shape=[], name="condition")
cost_jac_xq_last = tf.placeholder(tf.float32, shape=[],name='cost_jac_xq_last')
cost_jac_xy_last = tf.placeholder(tf.float32, shape=[],name='cost_jac_xy_last')
cost_jac_smooth1 = tf.cond(condition >0, 
                           lambda: lmbda_jac[1]* cost_jac_xq_last + (1-lmbda_jac[1])*cost_jac_xq ,
                           lambda: cost_jac_xq)
cost_jac_smooth2 = tf.cond(condition >0, 
                           lambda: lmbda_jac[1]* cost_jac_xy_last + (1-lmbda_jac[1])*cost_jac_xy ,
                           lambda: cost_jac_xy)
cost_jac_xq_smooth =  tf.identity(cost_jac_smooth1, name="cost_jac_xq_smooth")
cost_jac_xy_smooth =  tf.identity(cost_jac_smooth2, name="cost_jac_xy_smooth")

reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

# total cost
cost_xq = cost_pred_xq + lmbda_jac[0] * cost_jac_xq_smooth + 0.3*tf.add_n(reg_loss)
cost_xy = cost_pred_xy + lmbda_jac[3] * cost_jac_xy_smooth
cost =   lmbda_jac[2]* cost_xq  + cost_xy
# cost =    cost_xy

# cost = cost_pred 

In [ ]:
# gradient for velocity evaluation
g_forward1 = tf.gradients(y_nn, x_ph)
g_forward = tf.identity(g_forward1, name="g_forward")

g_backward1 = tf.gradients(x_rec, y_in)
g_backward = tf.identity(g_backward1, name="g_backward")

def f_grad_forward(_x):
    g = sess.run(g_forward,{x_ph: _x})
    return g
def f_g_backward(_x):
    g = sess.run(g_backward,{y_in: _x})
    return g

In [ ]:
# evaluation from x to quaternion offset
def f_x2q(_x):
     return sess.run(q_pred, {x_ph: _x})

def f_x2y(_x):
     return sess.run(y_nn, {x_ph: _x})
def f_y2x(_x):
     return sess.run(x_rec, {y_in: _x})

In [ ]:
rate = tf.Variable(0.1)
optimizer = tf.train.AdamOptimizer(rate)

train = optimizer.minimize(    cost, )
# initialize all variables to train
sess.run(tf.global_variables_initializer())

In [ ]:
start_time = time.time()

In [ ]:

_cost_jac_xq_smooth = 0.
_cost_jac_xy_smooth = 0.
xk_w = generate_pred_weight(nbPts, 100, 5.) 
xk_w = np.ones(sp*5)
delta_t = 0; t1 = 0
for i in range(500000):
    s, w = tmp_create_samples(x_data[:,:2],jac_samples_num)
    delta_t = time.time()-start_time - t1
    t1 = time.time()-start_time
    time_now =  int(t1)
    try:
        feed_dict = {
            x_ph : x_data[:,:2], # feed the actual points
            y_ph: y_data[:,:2],
            xk_weight: xk_w, 
            q_ph: delta_q , # feed their delta quaternions
            rate: 0.001, # rate of training
            condition: i,
            x_jac:s,
            weight_jac: w,
            cost_jac_xq_last: _cost_jac_xq_smooth,
            cost_jac_xy_last: _cost_jac_xy_smooth,
            lmbda_jac:  [1e-3, 0.0, 1, 2e-3  ],  # [xq_jac_w, smooth, xq weight, xy_jac_w ]            
        }
        _, _cost, _cost_pred_xq,_cost_xq, _cost_jac_xq_smooth,_cost_pred_xy,_cost_xy, _cost_jac_xy_smooth  = sess.run(
            [train, cost, cost_pred_xq,cost_xq, cost_jac_xq_smooth,cost_pred_xy,cost_xy, cost_jac_xy_smooth],  feed_dict)
#         xk2_pred = f_x2y(xk[:,:2])
        y_data_pred = f_x2y(x_data[:,:2])
        xk_pred_error = np.mean(np.sqrt(  np.sum( ((y_data_pred - y_data[:,:2])**2) , axis=1)   ))
        q_eva = f_x2q(x_data[:,:2])
        q_error = tools.ori_dis_np(q_eva, delta_q)
        if xk_pred_error<0.008 and np.mean(q_error * 180/np.pi)<4.8:
            break
        if i%20==0:
                display.clear_output(wait=True)
                print("time_now=", time_now,' delta_t', delta_t,'  i=',i, ' cost:',_cost, ' xk_error=',xk_pred_error)
                print('cost_pred_xq: ', _cost_pred_xq, '  cost_xq: ',_cost_xq, ' cost_jac_xq_smooth: ',_cost_jac_xq_smooth)
                print('cost_pred_xy: ', _cost_pred_xy, '  cost_xy: ',_cost_xy, ' q_error in degree: ',np.mean(q_error * 180/np.pi))
    except KeyboardInterrupt:
        break

In [ ]:
sp

In [ ]:

# saver = tf.train.Saver()
# save_path = './checkpoint_dir/MyModel_2D_iNN_sp'+str(sp)
# saver.save(sess, save_path)

In [ ]:
q_eva = f_x2q(x_data[:,:2])
q_error = tools.ori_dis_np(q_eva, delta_q)
print  'q_error(degree) mean and std: ',np.mean(q_error * 180/np.pi),  np.std(q_error * 180/np.pi)

xk2_pred = f_x2y(xk[:,:2])
xk_pred_error = np.sqrt(  np.sum( ((xk2_pred - xk2[:,:2])**2) , axis=1)   )
print  'xk mean and std: ',np.mean(xk_pred_error),  np.std(xk_pred_error)



In [ ]:
# %timeit f_grad_forward(xk[0:1,:2])    

In [ ]:
# %timeit f_g_backward(xk2[0:1,:2])

In [ ]:
# %timeit f_x2y(xk[0:1,:2])

In [ ]:
# %timeit f_y2x(xk2[0:1,:2])

In [ ]:
# %timeit q_eva = f_x2q(x_data[0:1,:2])


In [ ]:
def plot_2D_pos_ori(ax, data,color='r',alpha=1,length=0.01,head_width=0.007,lw=1,down_sampling=1,zorder =0):
    # data: [n, 6]
    arrow_len = np.array([length, 0, 0])
    if data.ndim==1:
        data = data.reshape(-1,1)        
    for i in range(data.shape[0]):
        x, y, q = data[i,0], data[i,1], quaternion.from_float_array(data[i,2:])
        new_arrow = quaternion.rotate_vectors(q, arrow_len) 
        tmp =  length* np.array([np.cos(data[i,2]), np.sin(data[i,2])])
#         print tmp
        ax.arrow(x, y, new_arrow[0],new_arrow[1],width=0.004,head_width=head_width,color=color,alpha=alpha,lw=lw,zorder=zorder)     


In [ ]:
red_color = np.array([228,73,26,255])/255.
green_color = np.array([100,165,74,255])/255.
xk_color = np.array([[38, 70, 83,255],
                    [42, 157, 143,255],
                    [242, 220, 166,255],
                    [243, 155, 83,255],
                    [197, 61, 27,255]])/255.

In [ ]:
plot_list = []
for i in range(0,nbGrid,downsampling):
    plot_list += range(i*nbGrid, (i+1)*nbGrid,downsampling )

grid_lines = generate_grid(xk, xk2)
nbGrid = int(np.sqrt(grid_lines.shape[0]))


grid_l = np.concatenate([grid_lines, np.repeat(np.array([[1.,0,0,0]]),nbGrid**2,axis=0 )   ],axis=1)
grid_l2 = np.concatenate([f_x2y(grid_lines[:,:2]), f_x2q(grid_lines[:,:2])], axis=1)


# fig, ax = plt.subplots(ncols=2, figsize=(fig_width, fig_width *4/8))
# color_xk = np.array([[0,1,0,1],[0.5,0.5,0,1],[0,0,1,1],[1,0,0,1],[1,0.5,0.5,1]])
# plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "8"
# plt.rcParams["text.usetex"] = True
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42


In [ ]:
%matplotlib inline
# fig, ax = plt.subplots(ncols=2, figsize=(fig_width, fig_width/2 *4/5))
# fig.subplots_adjust(left=0, bottom=0, right=1, top=1,
#                 wspace=0.33, hspace=0)
plot_list = []
for i in range(0,nbGrid,downsampling):
    plot_list += range(i*nbGrid, (i+1)*nbGrid,downsampling )
fig_width = 85/25.4 *8/9*2 # mm to inch
grid_lines = generate_grid(xk, xk2)
nbGrid = int(np.sqrt(grid_lines.shape[0]))


grid_l = np.concatenate([grid_lines, np.repeat(np.array([[1.,0,0,0]]),nbGrid**2,axis=0 )   ],axis=1)
grid_l2 = np.concatenate([f_x2y(grid_lines[:,:2]), f_x2q(grid_lines[:,:2])], axis=1)
grids1 = grid_l[:,:2].reshape(nbGrid,nbGrid,2)
grids2 = grid_l2[:,:2].reshape(nbGrid,nbGrid,2)

rate = 1.
fig0 = plt.figure(figsize=(fig_width/3*rate, fig_width/3 *4/5*rate))
ax0 = fig0.add_subplot(111)
fig1 = plt.figure(figsize=(fig_width/3, fig_width/3 *4/5))

ax1 = fig1.add_subplot(111, sharey=ax0)
ax = [ax0,ax1]
set_alpha = 1
grid_color = 'lightgray'
for i in range(0, nbGrid, downsampling):
    ax[0].plot(grids1[i,:,0], grids1[i,:,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
    ax[0].plot(grids1[:,i,0], grids1[:,i,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
    ax[1].plot(grids2[i,:,0], grids2[i,:,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
    ax[1].plot(grids2[:,i,0], grids2[:,i,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
# Set the zorder for the artist. Artists with lower zorder values are drawn first.

# ax[0].axis('equal');ax[1].axis('equal')
# xk_color = np.array([[]])
# ax[0].scatter(xk[:,0],xk[:,1],s = 10, c=xk_color,zorder = 10)
# ax[1].scatter(xk2[:,0],xk2[:,1],s = 10,c=xk_color,zorder = 10)
# for i in range(num_points):
#     ax[0].scatter(xk[i,0],xk[i,1],s = 15, c=xk_color[i,:],zorder = 20,marker=(4,0,45))
#     ax[1].scatter(xk2[i,0],xk2[i,1],s = 15,c=xk_color[i,:],zorder = 20,marker=(4,0,45+ right_point[i,3]-left_point[i,3]))
# y_pred = f_x2y(x_data[:,:2])



# for i in range(num_points):
#     ax[0].text(xk[i,0],xk[i,1],str(i+1),fontsize=10, zorder=100)
#     ax[1].text(xk2[i,0],xk2[i,1],str(i+1),fontsize=10, zorder=100 )

# ax[0].scatter(x_data[:,0],x_data[:,1],s=0.6, c='k')
# ax[1].scatter(y_pred[:,0],y_pred[:,1],s=0.6,c='k')
num_points = xk.shape[0]
xk_i = np.concatenate([xk[:,:2], np.repeat(np.array([[1.,0,0,0]]),num_points ,axis=0 )   ],axis=1)


# plot_2D_pos_ori(ax[0],grid_l[plot_list,:],color='gray',alpha=set_alpha,lw=0.5)
plot_2D_pos_ori(ax[1],grid_l2[plot_list,:],color='gray',alpha=set_alpha,lw=0.2,
                head_width=0.02,length=0.05,zorder = 5)
plot_2D_pos_ori(ax[0],grid_l[plot_list,:],color='gray',alpha=set_alpha,lw=0.2,
                head_width=0.02,length=0.05,zorder = 5)
# plot_2D_pos_ori(ax[1],grid_l2[plot_list,:],color='gray',alpha=set_alpha,lw=0.2,head_width=0.01,length=0.015,zorder = 5)
# plot_2D_pos_ori(ax[0],grid_l[plot_list,:],color='gray',alpha=set_alpha,lw=0.2,head_width=0.01,length=0.015,zorder = 5)
# plot_2D_pos_ori(ax[1], xk_tmp, color=red_color,length=0.05,zorder = 15,lw=0.2,head_width=0.02)  

# plot_2D_pos_ori(ax[0], xk_i, color=red_color,length=0.05,zorder = 15,lw=0.2,head_width=0.02)  
# ax[1].set_xticks([0,0.25,0.5],())
# ax[0].set_xticks([0,0.25,0.5],())
# ax[0].set_xlim(-0.1,0.75)
# ax[0].set_xticks([0,0.2,0.4,0.6],())

# ax[0].scatter(x_data[:,0], x_data[:,1],s=1)
# ax[1].scatter(y_data[:,0], y_data[:,1],s=1)
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "8"
# plt.rcParams["text.usetex"] = True
i=0
for data in x_key:
    ax[0].plot(data[:,0],data[:,1],c=xk_color[i,:],linewidth=1)
    i = i+1
i=0
for data in y_key:
    ax[1].plot(data[:,0],data[:,1],c=xk_color[i,:],linewidth=1)
    i = i+1
for i in range(2):
    ax[i].patch.set_facecolor("white")    
    ax[i].axis('on')   
    ax[i].spines['right'].set_visible(True)
    ax[i].spines['right'].set_color('k')
    ax[i].spines['left'].set_color('k')
    ax[i].spines['top'].set_color('k')
    ax[i].spines['bottom'].set_color('k')

    ax[i].spines['top'].set_visible(True)           
#     ax[i].axis('equal')
    ax[i].grid(False)
    ax[i].tick_params(axis='x', colors='k')
    ax[i].tick_params(axis='y', colors='k')
    ax[i].set_xticks([0,1,2],())
    ax[i].set_yticks([0,1,2],())
    
x_lim = np.array([-0.5,2.5])
y_lim = np.array([-0.5,1.75])+0.05  
ax[0].set_xlim(x_lim)
ax[0].set_ylim(y_lim)

ax[0].set_aspect(1.)
ax[1].set_aspect(1.)
ax[1].set_xlim(x_lim)
ax[1].set_ylim(y_lim)

# fig0.savefig('figures/2D_local_sp100_iNN.pdf',format='pdf',bbox_inches='tight',  pad_inches=0.0)
# fig1.savefig('figures/2D_remote_sp100_iNN.pdf',format='pdf',bbox_inches='tight',  pad_inches=0.0)
             

In [ ]:
### for sp=1 plot
%matplotlib inline
# fig, ax = plt.subplots(ncols=2, figsize=(fig_width, fig_width/2 *4/5))
# fig.subplots_adjust(left=0, bottom=0, right=1, top=1,
#                 wspace=0.33, hspace=0)
if sp ==1:
    fig_width = 85/25.4 *8/9*2 # mm to inch

    rate = 1.
    fig0 = plt.figure(figsize=(fig_width/3*rate, fig_width/3 *4/5*rate))
    ax0 = fig0.add_subplot(111)
    fig1 = plt.figure(figsize=(fig_width/3, fig_width/3 *4/5))

    ax1 = fig1.add_subplot(111, sharey=ax0)
    ax = [ax0,ax1]
    set_alpha = 1
    grid_color = 'lightgray'
    for i in range(0, nbGrid, downsampling):
        ax[0].plot(grids1[i,:,0], grids1[i,:,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
        ax[0].plot(grids1[:,i,0], grids1[:,i,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
        ax[1].plot(grids2[i,:,0], grids2[i,:,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
        ax[1].plot(grids2[:,i,0], grids2[:,i,1], c = grid_color, alpha=set_alpha,linewidth=0.5,zorder = 0)
    # Set the zorder for the artist. Artists with lower zorder values are drawn first.

  
    for i in range(num_points):
        ax[0].scatter(xk[i,0],xk[i,1],s = 15, c=xk_color[i,:],zorder = 20,marker=(4,0,45))
        ax[1].scatter(xk2[i,0],xk2[i,1],s = 15,c=xk_color[i,:],zorder = 20,marker=(4,0,45- angle[i]))


    xk_i = np.concatenate([xk[:,:2], np.repeat(np.array([[1.,0,0,0]]),num_points ,axis=0 )   ],axis=1)


    # plot_2D_pos_ori(ax[0],grid_lines[plot_list,:],color='gray',alpha=set_alpha,lw=0.5)
    plot_2D_pos_ori(ax[1],grid_l2[plot_list,:],color='gray',alpha=set_alpha,lw=0.2,
                    head_width=0.02,length=0.05,zorder = 5)
    plot_2D_pos_ori(ax[0],grid_l[plot_list,:],color='gray',alpha=set_alpha,lw=0.2,
                    head_width=0.02,length=0.05,zorder = 5)

    # xk_tmp  = np.concatenate([mapping_forward_evaluation(xk[:,:2]), xk2[:,2:] ],axis=1)
    plot_2D_pos_ori(ax[1], xk2, color=red_color,length=0.15,zorder = 15,lw=0.5,head_width=0.03)  

    plot_2D_pos_ori(ax[0], xk_i, color=red_color,length=0.15,zorder = 15,lw=0.5,head_width=0.03)  
    # ax[1].set_xticks([0,0.25,0.5],())
    # ax[0].set_xticks([0,0.25,0.5],())
    # ax[0].set_xlim(-0.1,0.75)
    # ax[0].set_xticks([0,1],())    
    # ax[1].set_xticks([0,1,2],())
    for i in range(2):
        ax[i].patch.set_facecolor("white")    
        ax[i].axis('on')   
        ax[i].spines['right'].set_visible(True)
        ax[i].spines['right'].set_color('k')
        ax[i].spines['left'].set_color('k')
        ax[i].spines['top'].set_color('k')
        ax[i].spines['bottom'].set_color('k')

        ax[i].spines['top'].set_visible(True)           
        ax[i].axis('equal')
        ax[i].grid(False)
        ax[i].tick_params(axis='x', colors='k')
        ax[i].tick_params(axis='y', colors='k')
    ax[1].set_xticks([0,1,2],())
    ax[1].set_yticks([0,1,2],())
    ax[0].set_xticks([0,1],())
    ax[0].set_yticks([0,1],())

# fig0.savefig('figures/2D_local_sp1_iNN.pdf',format='pdf',bbox_inches='tight',  pad_inches=0.0)
# fig1.savefig('figures/2D_remote_sp1_iNN.pdf',format='pdf',bbox_inches='tight',  pad_inches=0.0)